In [1]:
from qbuki import *

# qbuki.operators

`qbuki.operators` provides a useful abstraction for dealing with collections of operators. For example, we can wrap a POVM:

In [2]:
E = Operators(rand_povm(2, 4))
E.dim()

2

We can treat an `E` like it's an array:

In [3]:
len(E)

4

In [4]:
E[0:2]

array([[[0.581+0.j   , 0.051+0.183j],
        [0.051-0.183j, 0.162+0.j   ]],

       [[0.107-0.j   , 0.014-0.048j],
        [0.014+0.048j, 0.211-0.j   ]]])

In [5]:
E[0] = np.eye(2)

In [6]:
np.array([e.trace() for e in E])

array([2.   +0.j, 0.318-0.j, 0.547-0.j, 0.392-0.j])

Or conveniently:

In [7]:
E.bias()

array([2.   +0.j, 0.318-0.j, 0.547-0.j, 0.392-0.j])

In [8]:
E = Operators(rand_povm(2, 4))
sum(E)

array([[1.+0.j, 0.-0.j],
       [0.+0.j, 1.+0.j]])

Indeed, `E.conj()` complex conjugates each element in each operator, and `E.T` transposes each operator individually.

`A @ B` pariwise multiplies elements of A and B.

`A + B` concatenates the two collections of matrices, one after the other.

`A & B` gives the tensor product of each element of A with each element of B.

`aA = Aa` gives a scalar multiplication of each operator, and `A/a` scalar division.

`A.upgrade(i, dims)` upgrades each element to be supported on the i'th tensor factor of a tensor product with dimensions `dims`.

It's often useful to work with the matrix whose columns are the vectorized operators in the collection:

In [9]:
E.flatten()

array([[ 0.067-0.j   ,  0.356+0.j   ,  0.345+0.j   ,  0.232-0.j   ],
       [ 0.054-0.012j, -0.24 -0.135j, -0.018-0.052j,  0.203+0.199j],
       [ 0.054+0.012j, -0.24 +0.135j, -0.018+0.052j,  0.203-0.199j],
       [ 0.099-0.j   ,  0.252+0.j   ,  0.268+0.j   ,  0.381-0.j   ]])

We can obtain the matrix of traces between elements of two sets of operators (the Gram matrix):

In [10]:
E^E

array([[0.021+0.j, 0.026-0.j, 0.049+0.j, 0.071+0.j],
       [0.026+0.j, 0.342+0.j, 0.213-0.j, 0.027+0.j],
       [0.049-0.j, 0.213+0.j, 0.197+0.j, 0.154+0.j],
       [0.071-0.j, 0.027-0.j, 0.154-0.j, 0.36 +0.j]])

As well as its spectral inverse:

In [11]:
~E^E

array([[ 574.151+0.j,  154.509-0.j, -320.288+0.j,   12.69 -0.j],
       [ 154.509+0.j,   74.736+0.j, -137.177+0.j,   22.783-0.j],
       [-320.288-0.j, -137.177-0.j,  264.716+0.j,  -40.14 +0.j],
       [  12.69 +0.j,   22.783+0.j,  -40.14 -0.j,   15.778+0.j]])

The two sets of operators need not be the same:

In [12]:
A = Operators(rand_povm(2, 4))
B = Operators(rand_povm(2, 4))
A^B

array([[0.026-0.j, 0.147+0.j, 0.181-0.j, 0.14 -0.j],
       [0.035-0.j, 0.076-0.j, 0.278-0.j, 0.191+0.j],
       [0.016+0.j, 0.246+0.j, 0.181+0.j, 0.171+0.j],
       [0.003-0.j, 0.04 -0.j, 0.162+0.j, 0.106+0.j]])

Similarly, we can obtain the matrix of traces between elements of two sets of operators, where the second set are divided out by their traces. If we're working with two POVM's, this gives us a conditional probability matrix. 

In [13]:
A|B

array([[0.322-0.j, 0.288-0.j, 0.226-0.j, 0.23 -0.j],
       [0.438-0.j, 0.15 -0.j, 0.347+0.j, 0.314-0.j],
       [0.198+0.j, 0.483+0.j, 0.226+0.j, 0.282+0.j],
       [0.042-0.j, 0.079-0.j, 0.202+0.j, 0.174+0.j]])

We can then obtain the so-called Born matrix, given by the spectral inverse of the conditional probability matrix for a repeated measurement of a reference apparatus:

In [14]:
R = Operators(sic_povm(2))
~R|R

array([[ 2.5+0.j, -0.5-0.j, -0.5+0.j, -0.5-0.j],
       [-0.5+0.j,  2.5-0.j, -0.5-0.j, -0.5+0.j],
       [-0.5+0.j, -0.5-0.j,  2.5+0.j, -0.5-0.j],
       [-0.5-0.j, -0.5+0.j, -0.5-0.j,  2.5+0.j]])

And thus calculate the quantumness with respect to some p-norm:

$$ ||I_n - \Phi||_p$$

In [15]:
pnorm(np.eye(4) - (~R|R), p=2)

3.4641016151377535

Of course, the post-measurement states need not be proportional to POVM elements:

In [62]:
R = Operators(sic_povm(2))
S = Operators(rand_povm(2))
pnorm(np.eye(4) - (~R|S), p=2)

13.263502155383078

Or:

In [63]:
quantumness(R, S, p=2)

13.263502155383078

We can expand an operator (for example, a density matrix) in terms of the elements of our collection. In the case of a POVM, this gives us the probabilities for the outcomes of the measurement: 

In [17]:
E = Operators(rand_povm(2))
rho = rand_dm(2); rho

array([[0.949+0.j   , 0.136+0.132j],
       [0.136-0.132j, 0.051+0.j   ]])

In [18]:
E << rho

array([[0.103+0.j],
       [0.47 +0.j],
       [0.222+0.j],
       [0.205+0.j]])

And of course, we can go in reverse:

In [19]:
E << (E << rho)

array([[0.949+0.j   , 0.136+0.132j],
       [0.136-0.132j, 0.051+0.j   ]])

Any set of operators that span the operator space will do. For example, the discrete displacement operators provide a unitary operator basis:

In [20]:
D = Operators(displacement_operators(2))
D^D

array([[2.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.-0.j, 2.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 2.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 2.+0.j]])

In [21]:
D << rho

array([[1.   +0.j],
       [0.897-0.j],
       [0.272+0.j],
       [0.264+0.j]])

In [22]:
D << (D << rho)

array([[0.949+0.j   , 0.136+0.132j],
       [0.136-0.132j, 0.051+0.j   ]])

We can now express the fundamental lesson of quantum mechanics in QBist terms. Suppose we have a reference measurement $A$, consisting of an informationally complete POVM and a set of post-measurement states $S$ that span the operator space as well. We also have a second measurement $B$. We can calculate the probabilities for the outcomes of $B$, supposing we've encoded our beliefs about a quantum system as a density matrix $\rho$, and we send $\rho$ into $A$ first, and then $B$. The probabilities are given by the classical law of total probability:

In [29]:
A = Operators(rand_povm(2, 4))
S = Operators(np.array([rand_dm(2) for i in range(4)]))
B = Operators(rand_povm(2, 4))
rho = rand_dm(2)

In [30]:
(B|S) @ (A << rho)

array([[0.338+0.j],
       [0.186+0.j],
       [0.313+0.j],
       [0.163+0.j]])

Or equivalently:

In [31]:
B << sum(p*S[i] for i, p in enumerate(A << rho))

array([[0.338+0.j],
       [0.186+0.j],
       [0.313+0.j],
       [0.163+0.j]])

But what if we instead send the system directly into $B$, without actually going into $A$ first? Can we calculate the probabilities for $B$ in terms of the probabilities for $A$? Yes! We generalize the law of total probability in an elegant way:

In [33]:
(B|S) @ (~A|S) @ (A << rho)

array([[0.311-0.j],
       [0.09 -0.j],
       [0.356-0.j],
       [0.243+0.j]])

In [34]:
B << rho

array([[0.311-0.j],
       [0.09 -0.j],
       [0.356-0.j],
       [0.243-0.j]])

It's also worth noting that we can recover the Hilbert-Schmidt inner product between operators in a similar way:

In [36]:
rho = rand_dm(2)
sigma = rand_dm(2)
(rho@sigma).trace()

(0.4421876293419963+0j)

In [39]:
(S << sigma).T @ (~R^S) @ (R << rho)

array([[0.442-0.j]])

Moving on, our operators could be Kraus operators representing a quantum channel:

In [40]:
K = Operators(rand_kraus(2, 4))
sum(K.conj().T @ K)

array([[ 1.+0.j, -0.-0.j],
       [-0.+0.j,  1.+0.j]])

We can obtain the superoperator corresponding to the channel: 

In [41]:
K.superoperator() @ rho.flatten()

array([ 0.54 +0.j   , -0.077-0.055j, -0.077+0.055j,  0.46 +0.j   ])

In [42]:
sum([k @ rho @ k.conj().T for k in K]).flatten()

array([ 0.54 +0.j   , -0.077-0.055j, -0.077+0.055j,  0.46 -0.j   ])

More simply, we can express this as:

In [43]:
K < rho

array([[ 0.54 +0.j   , -0.077-0.055j],
       [-0.077+0.055j,  0.46 +0.j   ]])

Moreover, we can apply the channel to each operator in a collection:

In [44]:
K < E

Indeed, we can use this to express the action of a channel in QBist terms. For example, unitary evolution:

In [46]:
U = Operators(rand_unitary(2))
R << (U < rho)

array([[0.252+0.j],
       [0.199+0.j],
       [0.075+0.j],
       [0.474+0.j]])

In [48]:
(R|(U < S)) @ (~R|S) @ (R << rho)

array([[0.252-0.j],
       [0.199+0.j],
       [0.075+0.j],
       [0.474-0.j]])

We can also obtain the so-called frame superoperator:

In [49]:
R // S

array([[ 0.664+0.j   , -0.196+0.09j , -0.196-0.09j ,  0.336+0.j   ],
       [-0.051+0.033j,  0.085+0.001j, -0.05 -0.073j,  0.051-0.033j],
       [-0.051-0.033j, -0.05 +0.073j,  0.085-0.001j,  0.051+0.033j],
       [ 0.582+0.j   , -0.219+0.011j, -0.219-0.011j,  0.418+0.j   ]])

And its spectral inverse:

In [50]:
~R // S

array([[ -6.465 -0.j   , -18.158+13.742j, -18.158-13.742j,
          7.465 +0.j   ],
       [-16.999 -9.775j, -40.965+12.981j, -21.277-41.326j,
         16.999 +9.775j],
       [-16.999 +9.775j, -21.277+41.326j, -40.965-12.981j,
         16.999 -9.775j],
       [ -9.334 -0.j   ,  -8.07  +9.835j,  -8.07  -9.835j,
         10.334 +0.j   ]])

We can use these to calculate dual elements. Or more directly:

In [53]:
R**S << rho

array([[-7.752-0.j],
       [ 3.511+0.j],
       [ 3.372-0.j],
       [ 1.869-0.j]])

In [54]:
(~R|S) @ (R << rho)

array([[-7.752+0.j],
       [ 3.511-0.j],
       [ 3.372-0.j],
       [ 1.869-0.j]])

In [57]:
~R|S

array([[-66.677+0.j,  42.387-0.j, -27.323+0.j,  14.159-0.j],
       [ 36.989-0.j, -18.528+0.j,  14.315-0.j, -12.719+0.j],
       [  9.268-0.j,  -9.209+0.j,   8.347-0.j,   0.762+0.j],
       [ 21.42 -0.j, -13.65 +0.j,   5.661-0.j,  -1.202+0.j]])

In [60]:
(R**S)^(S**R)

array([[-66.677-0.j,  42.387-0.j, -27.323-0.j,  14.159+0.j],
       [ 36.989+0.j, -18.528+0.j,  14.315+0.j, -12.719-0.j],
       [  9.268+0.j,  -9.209+0.j,   8.347+0.j,   0.762-0.j],
       [ 21.42 +0.j, -13.65 +0.j,   5.661+0.j,  -1.202-0.j]])

Finally, we can apply a function to each element of our collection:

In [78]:
R_ = R % (lambda O: np.eye(2)/8+ O/2)
sum(R_)

array([[1.+0.j, 0.-0.j],
       [0.+0.j, 1.+0.j]])